In [1]:
%load_ext autoreload
%autoreload 2
import TRITON_SWMM_toolkit.examples as examples
import TRITON_SWMM_toolkit.analysis as analysis
from TRITON_SWMM_toolkit.process_simulation import TRITONSWMM_sim_post_processing
import TRITON_SWMM_toolkit.process_simulation as ps
import TRITON_SWMM_toolkit.processing_analysis as pe
from TRITON_SWMM_toolkit.process_simulation import *
import TRITON_SWMM_toolkit.plot_utils as plot_utils
import TRITON_SWMM_toolkit.utils as utils 
import TRITON_SWMM_toolkit.run_simulation as run
import TRITON_SWMM_toolkit.sensitivity_analysis as snst
from TRITON_SWMM_toolkit.config import analysis_config
from TRITON_SWMM_toolkit.examples import GetTS_TestCases as tst
from importlib import reload
reload(examples)
reload(analysis)
reload(ps)
reload(pe)
reload(plot_utils)
reload(utils)
reload(snst)
rebuild_case = True


#%%%% multisim
# testcase = tst.retreive_norfolk_multi_sim_test_case(start_from_scratch=rebuild_case)
# self = testcase.system.analysis

#%%%% sensitivity analysis
testcase = tst.retreive_norfolk_cpu_config_sensitivity_case(start_from_scratch=rebuild_case)
self = testcase.system.analysis.sensitivity

#%%%% single sim
# testcase = tst.retreive_norfolk_single_sim_test_case(
#         start_from_scratch=rebuild_case
#     )
# self = testcase.system.analysis

# testcase.system.process_system_level_inputs(overwrite_if_exists=rebuild_case)
# self.compile_TRITON_SWMM(verbose = True)
# self.sensitivity.prepare_scenarios_in_each_subanalysis()
# self.prepare_all_scenarios()

In [3]:
self.compile_TRITON_SWMM()
assert self.log.TRITONSWMM_compiled_successfully.get()

TRITON-SWMM already compiled


In [5]:
prepare_scenario_launchers = self.retrieve_prepare_scenario_launchers(
        overwrite_scenario=True, verbose=True
    )
self.run_python_functions_concurrently(prepare_scenario_launchers)
assert self.log.all_scenarios_created.get()

Memory-based limit: 3 (available 4083 MiB, 1024 MiB per task)
Running 2 functions (max parallel = 1)
[Scenario 0] Launching subprocess: python -m TRITON_SWMM_toolkit.prepare_scenario_runner --event-iloc 0 --analysis-config /home/dcl3nd/dev/TRITON-SWMM_toolkit/test_data/norfolk_coastal_flooding/tests/multi_sim/multi_sim.yaml --system-config /home/dcl3nd/dev/TRITON-SWMM_toolkit/test_data/norfolk_coastal_flooding/tests/tests.yaml --overwrite-scenario
[Scenario 0] Subprocess completed successfully
Function 0: duration=5.63s, completed_at=5.63s
[Scenario 1] Launching subprocess: python -m TRITON_SWMM_toolkit.prepare_scenario_runner --event-iloc 1 --analysis-config /home/dcl3nd/dev/TRITON-SWMM_toolkit/test_data/norfolk_coastal_flooding/tests/multi_sim/multi_sim.yaml --system-config /home/dcl3nd/dev/TRITON-SWMM_toolkit/test_data/norfolk_coastal_flooding/tests/tests.yaml --overwrite-scenario
[Scenario 1] Subprocess completed successfully
Function 1: duration=5.46s, completed_at=11.09s


In [6]:
launch_functions = self._create_launchable_sims(
    pickup_where_leftoff=False, verbose=True
)
self.run_simulations_concurrently_on_desktop(launch_functions, verbose=True)

Memory-based limit: 3 (available 4057 MiB, 1024 MiB per task)
Running up to 1 simulations concurrently
Simulation finished: simulation completed
Simulation finished: simulation completed


['simulation completed', 'simulation completed']

In [8]:
scenario_timeseries_processing_launchers = (
        self.retreive_scenario_timeseries_processing_launchers()
    )
self.run_python_functions_concurrently(scenario_timeseries_processing_launchers)

Memory-based limit: 3 (available 4069 MiB, 1024 MiB per task)
Running 2 functions (max parallel = 1)
Function 0: duration=27.03s, completed_at=27.03s
Function 1: duration=27.25s, completed_at=54.28s


[0, 1]

In [9]:
success_processing = (
        self.log.all_TRITON_timeseries_processed.get()
        and self.log.all_SWMM_timeseries_processed.get()
    )
success_processing

True